## Library Installation

In [ ]:
!pip install openai langchain langchain_openai langchain-experimental langchain-community neo4j unstructured

## Imports

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import CharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from google.colab import userdata
import os

## Neo4J Access Parameters

In [ ]:
graph = Neo4jGraph(
    url = "neo4j+s://bfe69ea7.*********",
    username = "neo4j",
    password = "HOrQGwpF*****************************"
)

## OpenAI Access API and Chat Parameters

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_APIKEY")

In [ ]:
llm = ChatOpenAI(temperature = 0)
llm_transf = LLMGraphTransformer(llm = llm)

In [ ]:
loader = DirectoryLoader('/content/Textdata')

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000, chunk_overlap = 20
)

In [ ]:
documents = text_splitter.split_documents(loader.load())
for d in documents:
  del d.metadata['source']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
graph_documents = llm_transf.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[]
Relationships:[]


In [ ]:
graph.add_graph_documents(graph_documents)

In [ ]:
graph.query("""
MATCH (n:Person) RETURN n
""")

[{'n': {'id': 'Ned'}},
 {'n': {'id': 'Robert'}},
 {'n': {'id': 'Daenerys Targaryen'}},
 {'n': {'id': 'Jaime'}},
 {'n': {'id': 'Tyrion'}},
 {'n': {'id': 'Catelyn'}},
 {'n': {'id': 'Lysa Arryn'}},
 {'n': {'id': 'Viserys'}},
 {'n': {'id': 'Khal Drogo'}},
 {'n': {'id': 'Joffrey'}},
 {'n': {'id': 'Cersei'}},
 {'n': {'id': 'Jon'}},
 {'n': {'id': 'Stannis'}},
 {'n': {'id': 'Littlefinger'}},
 {'n': {'id': 'Tywin Lannister'}},
 {'n': {'id': 'Robb Stark'}},
 {'n': {'id': 'Arya'}},
 {'n': {'id': 'Sansa'}},
 {'n': {'id': 'Knight'}},
 {'n': {'id': 'Mercenary'}},
 {'n': {'id': 'Father'}},
 {'n': {'id': 'Brother'}},
 {'n': {'id': 'Friends'}},
 {'n': {'id': 'Drogo'}},
 {'n': {'id': 'Daenerys'}},
 {'n': {'id': 'Village Woman'}},
 {'n': {'id': 'Child'}},
 {'n': {'id': 'Ser Jorah Mormont'}},
 {'n': {'id': 'Warriors'}},
 {'n': {'id': 'Sick And Old'}}]

In [ ]:
graph.schema

'Node properties:\nBook {id: STRING}\nContinent {id: STRING}\nPerson {id: STRING}\nFamily {id: STRING}\nCastle {id: STRING}\nPosition {id: STRING}\nGroup {id: STRING}\nStructure {id: STRING}\nPeople {id: STRING}\nAnimal {id: STRING}\nEvent {id: STRING}\nDirection {id: STRING}\nCreature {id: STRING}\nObject {id: STRING}\nInterest {id: STRING}\nLocation {id: STRING}\nActivity {id: STRING}\nPlace {id: STRING}\nAction {id: STRING}\nRelationship properties:\n\nThe relationships:\n(:Book)-[:SETTING]->(:Continent)\n(:Person)-[:REQUEST_FOR_ASSISTANCE]->(:Person)\n(:Person)-[:VISIT]->(:Castle)\n(:Person)-[:VISIT]->(:Person)\n(:Person)-[:VISIT]->(:Structure)\n(:Person)-[:RELATIONSHIP]->(:Person)\n(:Person)-[:ARRIVAL_WITH_QUEEN]->(:Person)\n(:Person)-[:OFFER_FOR_POSITION]->(:Person)\n(:Person)-[:DESTINATION]->(:Location)\n(:Person)-[:PROMISE]->(:Event)\n(:Person)-[:ACCEPTANCE]->(:Position)\n(:Person)-[:DEPARTURE]->(:Castle)\n(:Person)-[:JOURNEY_START]->(:Direction)\n(:Person)-[:ARRIVAL]->(:Locati

## GraphCypherQAChain Usage

In [ ]:
from langchain.chains import GraphCypherQAChain
llm = ChatOpenAI(temperature = 0)
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True
)

In [ ]:
chain.invoke("Who are the different persons?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN p.id;
Full Context:
[{'p.id': 'Ned'}, {'p.id': 'Robert'}, {'p.id': 'Daenerys Targaryen'}, {'p.id': 'Jaime'}, {'p.id': 'Tyrion'}, {'p.id': 'Catelyn'}, {'p.id': 'Lysa Arryn'}, {'p.id': 'Viserys'}, {'p.id': 'Khal Drogo'}, {'p.id': 'Joffrey'}]

> Finished chain.


{'query': 'Who are the different persons?',
 'result': 'Ned, Robert, Daenerys Targaryen, Jaime, Tyrion, Catelyn, Lysa Arryn, Viserys, Khal Drogo, Joffrey'}